In [1]:
!pip install --quiet transformers seaborn datasets evaluate rouge_score nltk numpy spacy lexrank

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 95.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 66.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 269.3/269.3 kB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 18.3 MB/s eta 0:

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!cp "/content/drive/MyDrive/NIR/predicted_data_relevant_part.zip" /content/predicted_data_relevant_part.zip
!unzip /content/predicted_data_relevant_part.zip

Archive:  /content/predicted_data_relevant_part.zip
  inflating: content/predicted_data.csv  


In [4]:
import re
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from lexrank import LexRank
from spacy.lang.ru.stop_words import STOP_WORDS

import nltk
from nltk.tokenize import sent_tokenize
import numpy as np
from datasets import Dataset
from datasets import load_dataset, load_metric
from sklearn.model_selection import train_test_split
import transformers
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    DataCollatorForSeq2Seq,
    HfArgumentParser,
    MBart50TokenizerFast,
    MBartTokenizer,
    MBartTokenizerFast,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    set_seed,
)
import evaluate

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
data = pd.read_csv('/content/content/predicted_data.csv')
data.head()

,match_id,news_id,after_match,name,team1,team2,content,part_body,len_content,relevant_part
0,18925,7274964,0,Россия. Премьер-лига,Локомотив,Химки,23 минута. Родолфо после верховой борьбы с Ант...,Антипенко получил травму в матче с «Локомотиво...,40,0
1,18925,7274964,0,Россия. Премьер-лига,Локомотив,Химки,80 минута. По многочисленным просьбам посетите...,Антипенко получил травму в матче с «Локомотиво...,28,0
2,18925,7274151,0,Россия. Премьер-лига,Локомотив,Химки,83 минута. Дуймович приложился по ногам Никифо...,Дюрица получил травму в игре с «Химками». Защи...,53,0
3,18925,7274964,0,Россия. Премьер-лига,Локомотив,Химки,"26 минута. гол! ""ХИМКИИИИ! Йованович, замыкая ...",Антипенко получил травму в матче с «Локомотиво...,23,0
4,18925,7274151,0,Россия. Премьер-лига,Локомотив,Химки,63 минута. СЫЧЕЕЕВ! Удар Дмитрия с семи метров...,Дюрица получил травму в игре с «Химками». Защи...,62,0


In [6]:
tokenizer = nltk.data.load('tokenizers/punkt/PY3/russian.pickle')

def del_spam(example):
  res = ''
  for sentence in tokenizer.tokenize(example):
    r = re.search(r'(онлайн-трансляц.+|таблица|здесь.|sports.ru)', sentence.lower())
    if r is None:
      res += sentence + ' '
  return res

data['part_body'] = data['part_body'] .apply(del_spam)

##### Обобщающие новости

In [ ]:
from pprint import pprint
from IPython.display import clear_output 

indexes = data[(data['after_match'] == 1) & (data['relevant_part'] == 1)].sample(frac=1).index.tolist()
result_index_test = []

for index in indexes:
  if len(result_index_test) == 100:
    break
  c_row = data.loc[index]
  print(c_row['team1'], c_row['team2'])
  pprint(c_row['content'], width=150)
  print('-----------')
  pprint(c_row['part_body'], width=150)
  if int(input()) == 1:
    result_index_test.append(index)
  clear_output(True)

Ростов Амкар
('8 минута. голь очень грубо прыгает прямой ногой в Канга! 11 минута. голЬ! голь-голь-голь! 0:1! Идеальная контратака "Амкара"! Коломейцев умчался '
 'по флангу от Бордачева и выкатил удобный пас поляку. Тому оставалось только не промахнуться из центра штрафной. 20 минута. Черенчиков пришел за '
 'Бухаровым в центр поля, не успел стартовать и сбил ростовчанина. 23 минута. А у "Ростова" замена. Баштуш вместо Кулу. Не совсем понятно, чем это '
 'вызвано. 39 минута. "Амкар" тоже не захотел ждать до перерыва с заменой. Шаваев вместо Йовичича. 41 минута. Бухаров бьет сам - и гол! 1:1! '
 'Издевательски легко переиграл форвард Геруса. 56 минута. Огуде ловит четвертое предупреждение и пропустит следующий матч. 59 минута. Пикущак '
 'выходит вместо голя. Теперь "Амкар" играет с одним нападающим. 78 минута. Бухаров ограничился одним голом. Вместо него выходит Полоз. 87 минута. '
 'Черенчиков удаляется! Пнул по ногам Калачева и идет отдыхать. 90 минута. А выходит Мартин Якубко у "Ам

In [7]:
result_index_test =  [7432,
                      26440,
                      63851,
                      56723,
                      40262,
                      43356,
                      32652,
                      46597,
                      58334,
                      64072,
                      41729,
                      19266,
                      53071,
                      49981,
                      58170,
                      37058,
                      54894,
                      52251,
                      27151,
                      41716,
                      20653,
                      50245,
                      51156,
                      14194,
                      18231,
                      34506,
                      58070,
                      45204,
                      47664,
                      31198,
                      33115,
                      36500,
                      36805,
                      12840,
                      46945,
                      36389,
                      53215,
                      61430,
                      58443,
                      52174,
                      37052,
                      35919,
                      40433,
                      27427,
                      57340,
                      17415,
                      9600,
                      29710,
                      25335,
                      52906,
                      55383,
                      44856,
                      48691,
                      30822,
                      48431,
                      27657,
                      46308,
                      50412,
                      50619,
                      52995,
                      37253,
                      39169,
                      36492,
                      51242,
                      35629,
                      61765,
                      57463,
                      43457,
                      50687,
                      27798,
                      61481,
                      35746,
                      41742,
                      27959,
                      54130,
                      34479,
                      62458,
                      28439,
                      50057,
                      7488,
                      55715,
                      49136,
                      10506,
                      5567,
                      28928,
                      37379,
                      41580,
                      41425,
                      33368,
                      63540,
                      49414,
                      10542,
                      26358,
                      30889,
                      20259,
                      48982,
                      14588,
                      42614,
                      55554,
                      50747
                      ]

In [8]:
temp_data = data[(data['after_match'] == 1) & (data['relevant_part'] == 1)]
temp_data = temp_data[~temp_data.index.isin(result_index_test)]
temp_data

,match_id,news_id,after_match,name,team1,team2,content,part_body,len_content,relevant_part
185,19003,25255675,1,Россия. Премьер-лига,Химки,Крылья Советов,5 минута. Кулик зарабатывает желтую карточку в...,«Крылья Советов» прервали безвыигрышную серию....,399,1
186,19004,26869751,1,Россия. Премьер-лига,Крылья Советов,Ростов,"42 минута. Калачев решил, что ""мстя будет жест...",Игроки «Ростова» получили выходной. После гост...,516,1
211,19010,26994127,1,Россия. Премьер-лига,Динамо,Рубин,25 минута. Бухаров жестко подкатился под Уикши...,Биглов получил «тройку» за судейство матча «Ди...,420,1
235,19014,28460679,1,Россия. Премьер-лига,Локомотив,Кубань,"4 минута. Капитану ""Локо"" сразу дали понять, к...",Сычев забил 60-й гол в чемпионатах России. Нап...,168,1
310,19020,31530734,1,Россия. Премьер-лига,ЦСКА,Крылья Советов,46 минута. Гильерме! С дебютом! Вместо Краваль...,Гильерме сделал дубль в дебютном матче за ЦСКА...,259,1
...,...,...,...,...,...,...,...,...,...,...
65730,1104865,1040572086,1,Кубок Стэнли,Питтсбург,Сан-Хосе,13 минута. гол! Раст - 1:0! 6-й гол в плей-офф...,НХЛ. Кубок Стэнли. Форварды «Питтсбурга» Раст ...,284,1
65732,1104866,1040637171,1,Кубок Стэнли,Питтсбург,Сан-Хосе,29 минута. гол! Кессел - 1:0! 10-й гол в плей-...,НХЛ. Звено Хагелин – Бонино – Кессел суммарно ...,285,1
65733,1104866,1040621229,1,Кубок Стэнли,Питтсбург,Сан-Хосе,29 минута. гол! Кессел - 1:0! 10-й гол в плей-...,НХЛ. Кубок Стэнли. Финал. «Питтсбург» вновь по...,285,1
65734,1104866,1040636713,1,Кубок Стэнли,Питтсбург,Сан-Хосе,29 минута. гол! Кессел - 1:0! 10-й гол в плей-...,НХЛ. Форвард «Питтсбурга» Шери забил первый по...,285,1


In [9]:
columns = ['match_id', 'news_id', 'name', 'team1', 'team2', 'content', 'part_body']

X_train_am, X_eval_am = train_test_split(temp_data[columns].sample(frac=1, random_state=42), test_size=1_500, random_state=42)
Test_am = data[data.index.isin(result_index_test)][columns]

##### Новости в течение матча

In [ ]:
from pprint import pprint
from IPython.display import clear_output 

indexes = data[(data['after_match'] == 0) & (data['relevant_part'] == 1)].sample(frac=1).index.tolist()
result_index_test = []

for index in indexes:
  if len(result_index_test) == 100:
    break
  c_row = data.loc[index]
  print(c_row['team1'], c_row['team2'])
  pprint(c_row['content'], width=150)
  print('-----------')
  pprint(c_row['part_body'], width=150)
  if int(input()) == 1:
    result_index_test.append(index)
  clear_output(True)

ЦСКА Ростов
('41 минута. Пенальти в ворота "Ростова"! 41 минута. Плетикоса ошибся на выходе, не смог удержать мяч, и Торбинский сбивает Фернандеса. 41 минута. '
 '42 минута. гол! Натхо! 1:0! Израильтянин спокойно развел вратаря и мяч по углам. 42 минута. 44 минута. Божович удален со скамейки! Видимо сказал '
 'что-то нехорошее про арбитров. 44 минута. 45 минута. Три минуты добавлено к первому тайму. 45 минута. Плетикоса хорош! На выходе успевает '
 'опередить Еременко. 45 минута. И еще один момент! Стеночка Мусы и Тошича на входе в штрафную, но Зоран не успел прицелиться, его тут же атаковал '
 'защитник. 45 минута. Супермомент у Вернблума! Но швед головой из отличной позиции бьет прямо в Плетикосу. 45 минута. Перерыв. ЦСКА впереди '
 'благодаря голу Натхо с пенальти. 45 минута. Вот и Максим Бордачев подлил масла в огонь в перерыве: "В пользу ЦСКА каждую игру назначают пенальти. '
 'Это проблематично". 45 минута.')
-----------
('ЦСКА – «Ростов». Судья удалил Божовича. Главный тренер «Р

In [10]:
result_index_test = [27675,
 47592,
 33327,
 30066,
 19637,
 34605,
 51322,
 41810,
 3951,
 63361,
 31649,
 36117,
 12421,
 10889,
 29971,
 59656,
 3625,
 17503,
 30440,
 62340,
 44593,
 39851,
 18338,
 60624,
 65291,
 13582,
 12884,
 30590,
 61937,
 58408,
 12405,
 37893,
 33557,
 56405,
 28595,
 63004,
 54805,
 39960,
 58481,
 65262,
 28506,
 21073,
 15584,
 8,
 10014,
 11404,
 10701,
 57790,
 28124,
 50345,
 23555,
 40696,
 40177,
 50012,
 10240,
 47983,
 47877,
 7625,
 40101,
 65540,
 57613,
 29082,
 10001,
 18459,
 22936,
 35025,
 41690,
 35805,
 26497,
 38679,
 42089,
 27315,
 62050,
 32033,
 36614,
 65456,
 42333,
 63168,
 56794,
 50858,
 50973,
 358,
 59331,
 44902,
 47954,
 34567,
 38469,
 54769,
 15499,
 14986,
 38388,
 65377,
 47091,
 28265,
 63354,
 59896,
 29926,
 65381,
 26928,
 50095]

In [11]:
temp_data = data[(data['after_match'] == 0) & (data['relevant_part'] == 1)]
temp_data = temp_data[~temp_data.index.isin(result_index_test)]
temp_data

,match_id,news_id,after_match,name,team1,team2,content,part_body,len_content,relevant_part
11,18925,7274964,0,Россия. Премьер-лига,Локомотив,Химки,80 минута. По многочисленным просьбам посетите...,Антипенко получил травму в матче с «Локомотиво...,38,1
22,18935,7014478,0,перенесенные матчи,Зенит,Штутгарт,"44 минута. ""Зенит"" пытается играть впереди быс...",Хусти забил в первом официальном матче в соста...,243,1
24,18935,7014478,0,перенесенные матчи,Зенит,Штутгарт,1 минута. Игра началась. 2 минута. Судя по том...,Хусти забил в первом официальном матче в соста...,69,1
30,18956,7114149,0,перенесенные матчи,Штутгарт,Зенит,40 минута. Тасчи возвращается на поле с переби...,Семшов забил первый гол в составе «Зенита». По...,45,1
63,18984,22193831,0,Россия. Премьер-лига,Химки,Динамо,"33 минута. Эх, Кержаков возмущен тем, что арби...",Агьяр забил в дебютном матче в составе «Динамо...,39,1
...,...,...,...,...,...,...,...,...,...,...
65609,1100868,1039637562,0,Россия. Олимп Кубок России,ЦСКА,Зенит,34 минута. ПЕНАЛЬТИ! Шатов навернул пару зидан...,ЦСКА – «Зенит». Оланаре покинул поле на носилк...,76,1
65642,1101630,1040274629,0,Чемпионат мира,Финляндия,Россия,38 минута. Овечкин неудачное решение принимает...,Россия — США. Телегин может сыграть в матче. Н...,282,1
65675,1103163,1040272252,0,Англия. Кубок,Кристал Пэлас,Манчестер Юнайтед,"18 минута. Клаттенбург уже устал свистеть, но ...",«Кристал Пэлас» – «МЮ». Смоллинг удален с поля...,65,1
65719,1104865,1040593364,0,Кубок Стэнли,Питтсбург,Сан-Хосе,"57 минута. ""Питтсбург"" поддавливает в концовке...",НХЛ. Марло не будет дисквалифицирован за удар ...,321,1


In [12]:
X_train_im, X_eval_im = train_test_split(temp_data[columns].sample(frac=1, random_state=42), test_size=1_000, random_state=42)
Test_im = data[data.index.isin(result_index_test)][columns]

In [13]:
X_train_im.to_csv('X_train_im.csv', index=False, encoding='utf-8')
X_eval_im.to_csv('X_eval_im.csv', index=False, encoding='utf-8')
Test_im.to_csv('Test_im.csv', index=False, encoding='utf-8')

X_train_am.to_csv('X_train_am.csv', index=False, encoding='utf-8')
X_eval_am.to_csv('X_eval_am.csv', index=False, encoding='utf-8')
Test_am.to_csv('Test_am.csv', index=False, encoding='utf-8')

#### Baseline

In [ ]:
# Метрика
metric = evaluate.load("rouge")

def postprocess_text(preds, labels):
  preds = [pred.strip() for pred in preds]
  labels = [label.strip() for label in labels]

  # rougeLSum ожидает новую строку после каждого предложения
  preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
  labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

  return preds, labels

def compute_metrics(preds, labels):
  preds, labels = postprocess_text(preds, labels)

  result = metric.compute(predictions=preds, references=labels, tokenizer=lambda x: x.split())
  result = {k: round(v * 100, 4) for k, v in result.items()}

  return result

##### Обобщающие новости

###### Eval

In [ ]:
lxr = LexRank(X_eval_am['content'].values, stopwords=STOP_WORDS)
X_test_summarize = X_eval_am['content'].apply(lambda x: ' '.join(lxr.get_summary(sent_tokenize(x), summary_size=4)))

In [ ]:
result = compute_metrics(X_test_summarize, X_eval_am['part_body'].apply(lambda x: x.lower()))
result

{'rouge1': 5.9874, 'rouge2': 0.133, 'rougeL': 4.9535, 'rougeLsum': 5.6373}

In [ ]:
# Пример суммаризации 1
print('Новость: ', X_eval_am.iloc[0]['part_body'])
print('Текст: ', X_eval_am.iloc[0]['content'])
print('Сгенерированная новость: ', X_test_summarize.iloc[0])

Новость:  «Крылья Советов» – «Ростов» – 0:2. Голы Канга и Дьякова принесли гостям победу. Сегодня в рамках 21-го тура российской премьер-лиги на стадионе «Металлург» в Самаре «Крылья Советов» сыграют с «Ростовом» из Ростова-на Дону. Sports.ru проведет онлайн-трансляцию этого матча . Начало – в 13.30 по московскому времени. Таблица премьер-лиги Статистика премьер-лиги
Текст:  13 минута. Первый "горчичник" в нашей игре. Форбс наказан за грубый фол на Жано. 24 минута. Втоую желтую карточку в игре получает Аджинджал. Хоть какое-то событие в сегодняшнем матче. 34 минута. Пришла очередь Логашова получить свою желтую карточку за срыв атаки. 35 минута. гол! 0:1 — Канга! Драгун не может, Канга поможет! На левом углу штрафной габонец неожиданно выстрелил в дальнюю девятку, чем серьезно удивил Веремко! Дальние удары сегодня в тренде! 52 минута. Воспитательные беседы рефери с Горо в первом тайме не пошли на пользу футболисту. За грубейший подкат против Канга хавбек самарцев получает желтую карточк

In [ ]:
# Пример суммаризации 2
print('Новость: ', X_eval_am.iloc[1]['part_body'])
print('Текст: ', X_eval_am.iloc[1]['content'])
print('Сгенерированная новость: ', X_test_summarize.iloc[1])

Новость:  «Зенит» – «Урал» – 2:1. Гол Ломбертса принес победу хозяевам. В 19-м туре чемпионата России «Зенит» на своем поле одолел «Урал» (2:1). Sports.ru провел онлайн-трансляцию матча «Зенит» – «Урал» . Таблица премьер-лиги Статистика премьер-лиги
Текст:  12 минута. гол! Заигрались гости в атаке и сразу же получили гол в свои ворота. Кришито по левому краю вошел в штрафную, выполнил прострел, Тумасян как-то неуверенно попытался его заблокировать, но в результате мяч оказался у Халка, который и отправил снаряд в сетку! 53 минута. Первая замена в матче. Вместо Быстрова на поле вышел Ансальди. 54 минута. гол! "Урал" забил хитрейший гол! Губочан на линии штрафной грубо встретил Ерохина, судья зафиксировал фол, но не показал жест о розыгрыше по свистку. Пока зенитовцы готовились к стандарту, Ерохин разыграл мяч с точки, отдав его Гогниеву, тот выскочил один на один с Лодыгиным и переиграл его. 1:1! 60 минута. Безысходность по-итальянски: Спаллетти бросает в игру не до конца выздоровевшего

In [ ]:
# Пример суммаризации 3
print('Новость: ', X_eval_am.iloc[2]['part_body'])
print('Текст: ', X_eval_am.iloc[2]['content'])
print('Сгенерированная новость: ', X_test_summarize.iloc[2])

Новость:  «Реал» – «Валенсия» – 3:2. Дубль Криштиану Роналду принес победу мадридцам. Сегодня в 37-м туре чемпионата Испании мадридский «Реал» одолел «Валенсию» на стадионе «Сантьяго Бернабеу» (3:2). Sports.ru проводил онлайн-трансляцию матча «Реал» – «Валенсия» . ПРИМЕЧАНИЕ: время начала матчей – московское. Таблица примеры Статистика примеры ВИДЕО
Текст:  13 минута. Энцо Перес жестко врезался в Хамеса, и следующий матч будет вынужден пропустить. 26 минута. гол! А тут уже Роналду сильнее вратаря! Сместился он чуть правее центра и неотразимым ударом попал в дальний угол ворот. Абсолютно типичный гол для португальца. 42 минута. гол! Бензема удваивает преимущество "Реала"! В первый раз не увидел судья офсайда у француза после передачи от Роналду, но атака не захлебнулась и уже во второй раз после передачи Канселу (!) Бензема спокойно в ближнем бою переиграл Алвеса. 55 минута. гол "Валенсии"! Родриго расстрелял практически пустые ворота! 2:1! 58 минута. КРИШТИАНУ РОНАЛДУ! 3:1! 33-й гол по

###### Test

In [ ]:
lxr = LexRank(Test_am['content'].values, stopwords=STOP_WORDS)
X_test_summarize = Test_am['content'].apply(lambda x: ' '.join(lxr.get_summary(sent_tokenize(x), summary_size=4)))

In [ ]:
result = compute_metrics(X_test_summarize, Test_am['part_body'].apply(lambda x: x.lower()))
result

{'rouge1': 5.7361, 'rouge2': 0.053, 'rougeL': 4.6448, 'rougeLsum': 5.2936}

In [ ]:
# Пример суммаризации 1
print('Новость: ', Test_am.iloc[0]['part_body'])
print('Текст: ', Test_am.iloc[0]['content'])
print('Сгенерированная новость: ', X_test_summarize.iloc[0])

Новость:  НХЛ. Плей-офф. 1/2 финала конференций. Шестой и седьмой матчи. «Монреаль» обыграл «Питтсбург», «Филадельфия» одолела «Бостон». Плей-офф 1/2 финала конференций Восточная конференция Седьмой матч Счет в серии: 3-4. Шестой матч Счет в серии: 3-3. ПРИМЕЧАНИЕ: время начала матчей – московское.
Текст:  1 минута. ...и уже на 10-й секунде отправил Джорджеса в борт. 2 минуты за бординг! 1 минута. Что происходит с пингвинами?! гол! Джионта подправил из района пятка после наброса слева Саббэна! 1 минута. Еще один пенальти у хозяев! Кук за игру высоко поднятой! 8 минута. Мур удален за неправильную атаку Гончара. 15 минута. 0:2! Мур! Пока там Орпик с Лапьером возился, так вот получилось. Брукси хотя бы ворота что ли сдвинул. А еще косвенно Кросби виноват оказался: неудачно выбросил шайбу из правого угла своей зоны, и после рикошета она отскочила в центр зоны к Муру, который бросил кистями, развернувшись - а такой бросок всегда непредсказуем. Можно сказать, что Доминику повезло с "мертвым"

In [ ]:
# Пример суммаризации 2
print('Новость: ', Test_am.iloc[1]['part_body'])
print('Текст: ', Test_am.iloc[1]['content'])
print('Сгенерированная новость: ', X_test_summarize.iloc[1])

Новость:  Пандев получил красную карточку по окончании матча с Россией. Нападающий «Наполи» и сборной Македонии Горан Пандев получил красную карточку после окончания матча отборочного турнира Евро-2012 с Россией (0:1). Такое наказание последовало после того, как он начал предъявлять претензии главному судье Бюленту Йылдириму. Напомним, в первом тайме Пандев оказался на газоне в штрафной сборной России, но пенальти назначен не был. А в ответной атаке хозяева забили гол. Sports.ru вел текстовую трансляцию матча Россия – Македония.
Текст:  41 минута. гол! 1:0! Семшов! Разыграли, Аршавин пролетел на скорости по центру, запустил налево Кержакову, тот заметил перед штрафной Семшова, который пробил прямо по центру, но небольшой рикошет от защитника способствовал тому, что голкипер соперников среагировал поздновато, от его перчаток мяч влетел в ворота. 42 минута. Пандев получил карточку. Скорее всего, за разговоры. 46 минута. Павлюченко вместо Семшова. 55 минута. Денисов получил карточку, проп

In [ ]:
# Пример суммаризации 3
print('Новость: ', Test_am.iloc[2]['part_body'])
print('Текст: ', Test_am.iloc[2]['content'])
print('Сгенерированная новость: ', X_test_summarize.iloc[2])

Новость:  Англия пробилась в финальную стадию Евро-2012. Сборная Англии, сыграв вничью с Черногорией (2:2), пробилась в финальную стадию Евро-2012. Подопечные Фабио Капелло завершили групповой этап с 18 очками. Это на 6 больше, чем у Черногории притом, что подопечным Бранко Брновича осталось провести только 1 матч. Отметим, что Черногория гарантировала себе 2-е место в группе и участие в стыковых матчах. Добавим, что Англия возвращается на Евро спустя 8 лет. Напомним, что они не смогли отобраться на Евро-2008, заняв в группе 3-е место и пропустив вперед Хорватию и Россию.
Текст:  10 минута. гол! 0:1! Руни, находясь в центре, отдал направо Уолкотту, тот прострелил в штрафную - и Эшли Янг, согнувшись, беспрепятственно пробил головой. Никто его не держал. 31 минута. гол! 0:2 - Бент. Эшли Янг набежал по правому флангу, хотел, видимо, сам пробить, но чуть промахнулся - недалеко от штанги мяч подобрал Даррен, и Младен Божович ничего не смог сделать. 45 минута. гол! Зверотич сокращает отрыв -

##### Новости в течение матча

###### Eval

In [ ]:
lxr = LexRank(X_train_im['content'].values, stopwords=STOP_WORDS)
X_test_summarize = X_train_im['content'].apply(lambda x: ' '.join(lxr.get_summary(sent_tokenize(x), summary_size=4)))

In [ ]:
result = compute_metrics(X_test_summarize, X_train_im['part_body'].apply(lambda x: x.lower()))
result

{'rouge1': 5.1988, 'rouge2': 0.1898, 'rougeL': 4.2102, 'rougeLsum': 4.9607}

In [ ]:
# Пример суммаризации 1
print('Новость: ', X_train_im.iloc[0]['part_body'])
print('Текст: ', X_train_im.iloc[0]['content'])
print('Сгенерированная новость: ', X_test_summarize.iloc[0])

Новость:  «МЮ» – «КПР». Ди Мария и Андер Эррера забили первые голы за манкунианцев. Вингер «Манчестер Юнайтед» Анхель Ди Мария открыл счет в матче 4-го тура АПЛ против «Куинз Парк Рейнджерс» (1:0, первый тайм). На 24-й минуте футболист сборной Аргентины издалека навесил со штрафного удара, и мяч влетел в сетку после того, как никто из игроков обеих команд его не коснулся. Таким образом, Ди Мария забил первый гол за манкунианцев. Кроме того, на 36-й минуте встречи свой первый гол за «МЮ» забил 25-летний испанский хавбек Андер Эррера с передачи Уэйна Руни. Напомним, что трансфер
Текст:  24 минута. гол! Диииииии Мария! Выполнил неудобную подачу со штрафного, до мяча никто не дотянулся, и тот влетел в дальний угол! 1:0! Первый гол аргентинца за "Юнайтед" состоялся. 24 минута. 26 минута. Хозяева не расслабляются и продолжают прессовать соперника. 27 минута. В штрафной падает Эррера, но Дауд не свистит. Был там контакт, но испанец и сам на него шел.
Сгенерированная новость:  Был там контакт,

In [ ]:
# Пример суммаризации 2
print('Новость: ', X_train_im.iloc[1]['part_body'])
print('Текст: ', X_train_im.iloc[1]['content'])
print('Сгенерированная новость: ', X_test_summarize.iloc[1])

Новость:  «Магнитка» – СКА. Коскинен отразил 86 процентов бросков. Голкипер СКА Микко Коскинен в матче с «Магниткой» (3:4 ОТ) пропустил 4 шайбы после 30 бросков по своим воротам. Таким образом, финн отразил 86,6 процента бросков. До 56-й минуты питерцы вели 3:1, после чего пропустили три гола подряд. В предыдущей игре с московским «Динамо» Коскинен отразил 16 бросков из 18 . Камбэк «Магнитки» в матче со СКА. Как это было
Текст:  56 минута. Гости даже особо не стремятся угрожать воротам, находясь в чужой зоне, просто держат шайбу. 57 минута. Коскинен вылавливает шайбу после наброса Коваржа на пятак. 57 минута. Но не тут-то было! гол! 2:3! Мозякин! Антипин бросал от синей, Мозякин подставлял и со второй попытки поразил ворота Коскинена! 58 минута. Армейцы тут же оживились, ведь у "Металлурга" появилась надежда! 59 минута. Вот это концовка! "Магнитка" вытаскивает матч! 59 минута. гол! 3:3! Зарипов при отложенном штрафе, 6 на 4! Бросок Коваржа из круга, и Зарипов преуспел на добивании! 60 

In [ ]:
# Пример суммаризации 3
print('Новость: ', X_train_im.iloc[2]['part_body'])
print('Текст: ', X_train_im.iloc[2]['content'])
print('Сгенерированная новость: ', X_test_summarize.iloc[2])

Новость:  «Динамо» – ЦСКА. Кураньи пропустит матч из-за перебора карточек. Форвард «Динамо» Кевин Кураньи не примет участия в матче 24-го тура премьер-лиги с ЦСКА. 31-летний нападающий получил четвертую желтую карточку во встрече 23-го тура чемпионата России с «Амкаром» (0:1, перерыв). В текущем сезоне на счету Кураньи 7 голов и 2 голевых передачи в 21 матче премьер-лиги. Подробную статистику выступлений игрока смотрите здесь . Матч «Динамо» – ЦСКА состоится 12 апреля в Химках.
Текст:  44 минута. Кураньи получил мяч во вратарской, замешкался, но со второй попытки все же пробил по воротам, однако от ноги защитника мяч ушел за лицевую! 44 минута. Футболисты "Динамо" взялись объяснить судье Безбородову некоторые аспекты футбольных правил и получили сразу две желтых карточки! Второе предупреждение в "Динамо" для Кевина Кураньи... 44 минута. ... и третье предупреждение в "Динамо" для Нобоа. 45 минута. А вот и четвертое предупреждение для "Динамо" : Ломич ударил Георгиева сзади по ногам. 45 

###### Test

In [ ]:
lxr = LexRank(Test_im['content'].values, stopwords=STOP_WORDS)
X_test_summarize = Test_im['content'].apply(lambda x: ' '.join(lxr.get_summary(sent_tokenize(x), summary_size=4)))

In [ ]:
result = compute_metrics(X_test_summarize, Test_im['part_body'].apply(lambda x: x.lower()))
result

{'rouge1': 5.9424, 'rouge2': 0.4592, 'rougeL': 4.8069, 'rougeLsum': 5.8115}

In [ ]:
# Пример суммаризации 1
print('Новость: ', Test_im.iloc[0]['part_body'])
print('Текст: ', Test_im.iloc[0]['content'])
print('Сгенерированная новость: ', X_test_summarize.iloc[0])

Новость:  Белоруков и Федорив получили красные карточки в игре с «Зенитом». Защитники «Амкара» Дмитрий Белоруков и Виталий Федорив были удалены с поля в игре 24-го тура премьер-лиги против «Зенита». На 42-й минуте игры Белоруков получил вторую желтую карточку за нарушение на Владимире Быстрове. На 55-й минуте также второе предупреждение было показано Виталию Федориву фол был совершен на Александре Анюкове. Текстовый онлайн встречи «Зенит» «Амкар» можно посмотреть здесь . P.S. Обновленная футбольная «Трибуна» : свободное общение + еще больше интересного чтения.
Текст:  55 минута. Турбин! Турбин жесткоковыйный! Удаляет Федорива за вторую карточку. Фол так себе был, конечно. И тем не менее. 56 минута. 9-11. Многозначительный расклад. 57 минута. А подачи у "Зенита", тем не менее, по-прежнему не наладились. 57 минута. Быстров падает в штрафной, не успевает судья задуматься о том, что делать - как Кержаков бьет "шведой" в ближнюю девятку. Мимо.
Сгенерированная новость:  Ближе всех был Бухаро

In [ ]:
# Пример суммаризации 2
print('Новость: ', Test_im.iloc[1]['part_body'])
print('Текст: ', Test_im.iloc[1]['content'])
print('Сгенерированная новость: ', X_test_summarize.iloc[1])

Новость:  Веллитон нанес травму Габулову. Голкипер «Динамо» Владимир Габулов получил травму в матче 21-го тура премьер-лиги против «Спартака». В середине первого тайма в него врезался нападающий красно-белых Веллитон и нанес голкиперу удар коленом в голову. У Габулова подозрение на перелом носа. Он был сразу же отправлен в больницу. За свое нарушение Веллитон получил желтую карточку. Место в воротах «Динамо» занял Антон Шунин.
Текст:  45 минута. Хохлов принимает мяч в штрафной, опускает его на землю, проходит еще тысяча пятнадцать мгновений, замахивается, и только тогда его успевают накрыть шустрые центрдефы "Спартака"! 45 минута. Уилкшир оттянулся на Веллитоне! Надо бы карточку дать! 45 минута. Алекс скатил под удар Жано. Ананидзе бьет и попадает в стенку! 45 минута. Перерыв! 45 минута. "Спартак" играет сегодня непонятно во что, хотя и контролирует мяч большую часть времени. "Динамо", может, тоже не отличается изысканностью комбинаций, но зато их атаки острее, да и гол уже есть в акти

In [ ]:
# Пример суммаризации 3
print('Новость: ', Test_im.iloc[2]['part_body'])
print('Текст: ', Test_im.iloc[2]['content'])
print('Сгенерированная новость: ', X_test_summarize.iloc[2])

Новость:  Белоруков и Федорив получили красные карточки в игре с «Зенитом». Защитники «Амкара» Дмитрий Белоруков и Виталий Федорив были удалены с поля в игре 24-го тура премьер-лиги против «Зенита». На 42-й минуте игры Белоруков получил вторую желтую карточку за нарушение на Владимире Быстрове. На 55-й минуте также второе предупреждение было показано Виталию Федориву фол был совершен на Александре Анюкове. Текстовый онлайн встречи «Зенит» «Амкар» можно посмотреть здесь . P.S. Обновленная футбольная «Трибуна» : свободное общение + еще больше интересного чтения.
Текст:  55 минута. Турбин! Турбин жесткоковыйный! Удаляет Федорива за вторую карточку. Фол так себе был, конечно. И тем не менее. 56 минута. 9-11. Многозначительный расклад. 57 минута. А подачи у "Зенита", тем не менее, по-прежнему не наладились. 57 минута. Быстров падает в штрафной, не успевает судья задуматься о том, что делать - как Кержаков бьет "шведой" в ближнюю девятку. Мимо.
Сгенерированная новость:  Мимо. Быстров падает 

### Новости в течение матча

#### Fine-tuning модели mbart-large-cc25

In [14]:
SEED = 3
lang = 'ru_RU'
set_seed(SEED)

In [15]:
dataset = load_dataset("csv", data_files={'train': 'X_train_im.csv', 'eval': 'X_eval_im.csv', 'test': 'Test_im.csv'})

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating eval split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/default-71c7c1f7b7e7830a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [16]:
dataset['train'][0]

{'match_id': 955545,
 'news_id': 1023566887,
 'name': 'Англия. Премьер-лига',
 'team1': 'Манчестер Юнайтед',
 'team2': 'КПР',
 'content': '24 минута. гол! Диииииии Мария! Выполнил неудобную подачу со штрафного, до мяча никто не дотянулся, и тот влетел в дальний угол! 1:0! Первый гол аргентинца за "Юнайтед" состоялся. 24 минута. 26 минута. Хозяева не расслабляются и продолжают прессовать соперника. 27 минута. В штрафной падает Эррера, но Дауд не свистит. Был там контакт, но испанец и сам на него шел.',
 'part_body': '«МЮ» – «КПР». Ди Мария и Андер Эррера забили первые голы за манкунианцев. Вингер «Манчестер Юнайтед» Анхель Ди Мария открыл счет в матче 4-го тура АПЛ против «Куинз Парк Рейнджерс» (1:0, первый тайм). На 24-й минуте футболист сборной Аргентины издалека навесил со штрафного удара, и мяч влетел в сетку после того, как никто из игроков обеих команд его не коснулся. Таким образом, Ди Мария забил первый гол за манкунианцев. Кроме того, на 36-й минуте встречи свой первый гол за «

In [17]:
model_name = "facebook/mbart-large-cc25"

config = transformers.AutoConfig.from_pretrained(model_name)

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForSeq2SeqLM.from_pretrained(model_name, config=config)

In [18]:
model.config.decoder_start_token_id = tokenizer.convert_tokens_to_ids(lang)

In [19]:
tokenizer.src_lang = lang
tokenizer.tgt_lang = lang

# Для mbart нужен токен языка
forced_bos_token_id = (tokenizer.lang_code_to_id[lang])

model.config.forced_bos_token_id = forced_bos_token_id

In [20]:
text_column = 'content'
summary_column = 'part_body'

In [21]:
max_source_length = 512
max_target_length = 300
padding = 'max_length'
ignore_pad_token_for_loss = True

In [24]:
def preprocess_function(examples):
  inputs, targets = [], []

  for i in range(len(examples[text_column])):
      if examples['name'][i] and examples['team1'][i] and examples['team2'][i] and examples[text_column][i] and examples[summary_column][i]:
          inputs.append('</s>'.join([examples['name'][i], examples['team1'][i], examples['team2'][i], examples[text_column][i]]))
          targets.append(examples[summary_column][i])

  model_inputs = tokenizer(inputs, max_length=max_source_length, padding=padding, truncation=True)

  labels = tokenizer(text_target=targets, max_length=max_target_length, padding=padding, truncation=True)

  # Для игнорирования паддинга в loss
  if padding == "max_length" and ignore_pad_token_for_loss:
      labels["input_ids"] = [
          [(l if l != tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
      ]

  model_inputs["labels"] = labels["input_ids"]

  return model_inputs

In [25]:
train_dataset = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=2,
    remove_columns=['match_id', 'news_id', 'name', 'team1', 'team2', 'content', 'part_body'],
    )

Map (num_proc=2):   0%|          | 0/6460 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/1000 [00:00<?, ? examples/s]

Map (num_proc=2):   0%|          | 0/100 [00:00<?, ? examples/s]

In [27]:
# Data collator
label_pad_token_id = -100 
data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=label_pad_token_id,
    pad_to_multiple_of=8
)

In [28]:
# Метрика
metric = evaluate.load("rouge")

In [29]:
def postprocess_text(preds, labels):
  preds = [pred.strip() for pred in preds]
  labels = [label.strip() for label in labels]

  # rougeLSum ожидает новую строку после каждого предложения
  preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
  labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

  return preds, labels

In [30]:
def compute_metrics(eval_preds):
  preds, labels = eval_preds
  decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
  if ignore_pad_token_for_loss:
      labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

  result = metric.compute(predictions=decoded_preds, references=decoded_labels, tokenizer=lambda x: x.split())
  result = {k: round(v * 100, 4) for k, v in result.items()}
  prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
  result["gen_len"] = np.mean(prediction_lens)
  return result

In [31]:
training_args = Seq2SeqTrainingArguments(
    output_dir=".",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
    fp16=True,
    optim="adafactor",
    logging_steps=100,
    evaluation_strategy='epoch',
    save_strategy="epoch",
    load_best_model_at_end=True,
    predict_with_generate=True
    )

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset['train'],
    eval_dataset=train_dataset['eval'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
    )

train_result = trainer.train()

You're using a MBartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
0,1.489900,1.252136,28.035500,12.223000,22.905100,26.731100,110.901000
1,1.113200,1.035859,29.770100,13.714700,24.253100,28.248200,112.647000
2,0.960000,0.965837,30.083900,13.918100,24.421200,28.637000,111.358000


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


In [39]:
!cp -r "/content/checkpoint-603" "/content/drive/MyDrive/NIR/mbart-im"

In [ ]:
trainer.save_model('/content/drive/MyDrive/NLP_course_work')

#### Оценка качества настроенного mBART на тестовом наборе данных

In [33]:
predict_results = trainer.predict(
    train_dataset['test'],
    metric_key_prefix="predict",
    max_length=max_target_length,
    num_beams=3
)
metrics = predict_results.metrics

In [34]:
metrics

{'predict_loss': 0.965757429599762,
 'predict_rouge1': 35.4406,
 'predict_rouge2': 17.889,
 'predict_rougeL': 28.9973,
 'predict_rougeLsum': 33.9969,
 'predict_gen_len': 106.87,
 'predict_runtime': 119.1651,
 'predict_samples_per_second': 0.839,
 'predict_steps_per_second': 0.21}

In [35]:
predictions = []
if trainer.is_world_process_zero():
  if training_args.predict_with_generate:
    predictions = tokenizer.batch_decode(
        predict_results.predictions, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )
    predictions = [pred.strip() for pred in predictions]

In [36]:
output_prediction_file = "generated_predictions.txt"
with open(output_prediction_file, "w", encoding='utf-8') as writer:
  writer.write("\n".join(predictions).encode('utf-8').decode("utf-8"))

In [37]:
print('часть матча: ', dataset['test'][0]['content'])
print('оригинальная новость: ', dataset['test'][0]['part_body'])
print('предсказанная новость: ', predictions[0])

часть матча:  44 минута. Травмированного Дюрицу меняет Фининьо. 45 минута. Муджири метров с 20 ковырнул мяч прямо в руки Березовскому. 45 минута. Перерыв. Непочатый край работы у Рахимова. 45 минута. PrisonerR, о травме в преддверии матча ничего не сообщалось. Низамутдинов в запасе. 45 минута. "Химки" реализовали свой единственный шанс, если опустить прерванный рейд Кожанова. Более того, в створ ворот Чеха был нанесен лишь один удар, который и обратился в гол. 45 минута. "Локомотив" выжал из себя одиннадцать ударов, однако в "единичку" на табло не обратился ни один из них. 46 минута. Второй тайм! Замен нет.
оригинальная новость:  Дюрица получил травму в игре с «Химками». В конце первого тайма его заменил Фининьо. Дюрица покинул поле и ушел в подтрибунное помещение, сильно прихрамывая и с ледяным компрессом на правой ноге. Тяжесть травмы словака пока неизвестна. 
предсказанная новость:  «Локомотив» – «Химки». Дюрицу получил травму. Защитник «Локомотива» Марио Дюрицу получил травму в мат

In [38]:
print('часть матча: ', dataset['test'][1]['content'])
print('оригинальная новость: ', dataset['test'][1]['part_body'])
print('предсказанная новость: ', predictions[1])

часть матча:  45 минута. Хохлов принимает мяч в штрафной, опускает его на землю, проходит еще тысяча пятнадцать мгновений, замахивается, и только тогда его успевают накрыть шустрые центрдефы "Спартака"! 45 минута. Уилкшир оттянулся на Веллитоне! Надо бы карточку дать! 45 минута. Алекс скатил под удар Жано. Ананидзе бьет и попадает в стенку! 45 минута. Перерыв! 45 минута. "Спартак" играет сегодня непонятно во что, хотя и контролирует мяч большую часть времени. "Динамо", может, тоже не отличается изысканностью комбинаций, но зато их атаки острее, да и гол уже есть в активе. 45 минута. Главным событием первого тайма стало столкновение Веллитона и Габулова, в результате которого у последнего подозрение на сотрясение мозга и перелом носа. Все-таки, игровой момент был - Веллитон травму нанес не специально, а за то, что пошел в опасное единоборство до конца, получил вполне заслуженную карточку. Такие эпизоды не редкость. Вон, давеча Павлюченко в Йелле ногой въехал. 46 минута. Баженов выходит 

In [39]:
print('часть матча: ', dataset['test'][2]['content'])
print('оригинальная новость: ', dataset['test'][2]['part_body'])
print('предсказанная новость: ', predictions[2])

часть матча:  55 минута. Турбин! Турбин жесткоковыйный! Удаляет Федорива за вторую карточку. Фол так себе был, конечно. И тем не менее. 56 минута. 9-11. Многозначительный расклад. 57 минута. А подачи у "Зенита", тем не менее, по-прежнему не наладились. 57 минута. Быстров падает в штрафной, не успевает судья задуматься о том, что делать - как Кержаков бьет "шведой" в ближнюю девятку. Мимо.
оригинальная новость:  Белоруков и Федорив получили красные карточки в игре с «Зенитом». Защитники «Амкара» Дмитрий Белоруков и Виталий Федорив были удалены с поля в игре 24-го тура премьер-лиги против «Зенита». На 42-й минуте игры Белоруков получил вторую желтую карточку за нарушение на Владимире Быстрове. На 55-й минуте также второе предупреждение было показано Виталию Федориву фол был совершен на Александре Анюкове. P.S. Обновленная футбольная «Трибуна» : свободное общение + еще больше интересного чтения. 
предсказанная новость:  «Зенит» – «Амкар». Турбин удален с поля. Полузащитник «Амкара» Алекса